In [1]:
import pandas as pd

In [2]:
ai_result_path = "./AI_Mapping_Result/V3/1_BAY_mapping_result_V3_rev6.csv"
prod_result_path = "./Listed_Mapping_Prod/BAY_Q2_2024.csv"

In [3]:
dtype = {
    'set_code': 'string',
}

ai_df = pd.read_csv(ai_result_path, encoding='utf-8', dtype=dtype)
ai_df.head()

,Unnamed: 0,query,set_code,item_in_taxonomy,reasoning,table_name,category,number_values
0,0,เงินสด,100100,เงินสด,The query 'เงินสด' matches directly with the '...,งบฐานะการเงิน,สินทรัพย์,"[24662778, 28169604, 22702027, 25127673]"
1,1,รายการระหว่างธนาคารและตลาดเงินสุทธิ,100200,รายการระหว่างธนาคารและตลาดเงิน - สุทธิ,The input name 'รายการระหว่างธนาคารและตลาดเงิน...,งบฐานะการเงิน,สินทรัพย์,"[524632695, 507974332, 501846916, 489172830]"
2,2,สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธ...,100600,สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธ...,The query 'สินทรัพย์ทางการเงินที่วัดมูลค่าด้วย...,งบฐานะการเงิน,สินทรัพย์,"[3345144, 1740461, 2303149, 622531]"
3,3,สินทรัพย์อนุพันธ์,103000,สินทรัพย์อนุพันธ์,The input item name is สินทรัพย์อนุพันธ์ and f...,งบฐานะการเงิน,สินทรัพย์,"[37369353, 29055607, 37583142, 29257667]"
4,4,เงินลงทุนสุทธิ,101600,เงินลงทุน - สุทธิ,The input query 'เงินลงทุนสุทธิ' is a close ma...,งบฐานะการเงิน,สินทรัพย์,"[137029713, 130538799, 131584335, 125353247]"


In [4]:
dtype = {
    'account_code': 'string',
}

prod_df = pd.read_csv(prod_result_path, encoding='utf-8', dtype=dtype)
prod_df.head()

,account_name,taxonomy_account_name,account_code,values
0,เงินสด,เงินสด,100100,"['24,662,778', '22,702,027']"
1,รายการระหว่างธนาคารและตลาดเงินสุทธิ,รายการระหว่างธนาคารและตลาดเงินสุทธิ,100200,"['524,632,695', '501,846,916']"
2,สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธ...,สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธ...,100690,"['3,345,144', '2,303,149']"
3,สินทรัพย์อนุพันธ์,สินทรัพย์อนุพันธ์,103000,"['37,369,353', '37,583,142']"
4,เงินลงทุนสุทธิ,เงินลงทุนสุทธิ,101600,"['137,029,713', '131,584,335']"


In [6]:
compared_list = []

for index, row in prod_df.iterrows():
    prod_name = row['taxonomy_account_name']
    prod_acc_code = row['account_code']
    # print(prod_name)
    # print(prod_acc_code, type(prod_acc_code))

    ai_row = ai_df[ai_df['set_code'] == prod_acc_code]
    # print(len(ai_row.index))

    ai_acc_code = ""
    if (len(ai_row.index) > 0):
        ai_acc_code = ai_row['set_code'].iloc[0]
    # print(ai_acc_code)

    compare_item = {
        "taxonomy_account_name": prod_name,
        "ai_account_code": ai_acc_code,
        "prod_account_code": prod_acc_code,
        "is_match": str(ai_acc_code) == str(prod_acc_code)
    }

    print(compare_item)
    compared_list.append(compare_item)

{'taxonomy_account_name': 'เงินสด', 'ai_account_code': '100100', 'prod_account_code': '100100', 'is_match': True}
{'taxonomy_account_name': 'รายการระหว่างธนาคารและตลาดเงินสุทธิ', 'ai_account_code': '100200', 'prod_account_code': '100200', 'is_match': True}
{'taxonomy_account_name': 'สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธรรมผ่านกำไรหรือขาดทุน', 'ai_account_code': '', 'prod_account_code': '100690', 'is_match': False}
{'taxonomy_account_name': 'สินทรัพย์อนุพันธ์', 'ai_account_code': '103000', 'prod_account_code': '103000', 'is_match': True}
{'taxonomy_account_name': 'เงินลงทุนสุทธิ', 'ai_account_code': '101600', 'prod_account_code': '101600', 'is_match': True}
{'taxonomy_account_name': 'เงินลงทุนในบริษัทย่อยบริษัทร่วมและการร่วมค้าสุทธิ', 'ai_account_code': '102600', 'prod_account_code': '102600', 'is_match': True}
{'taxonomy_account_name': 'เงินให้สินเชื่อแก่ลูกหนี้และดอกเบี้ยค้างรับสุทธิ', 'ai_account_code': '102000', 'prod_account_code': '102000', 'is_match': True}
{'taxonomy_ac

In [7]:
count_unmatch = 0
for item in compared_list:
    if (item['is_match'] == False):
        count_unmatch += 1
        print(item)

print(f"Total Unmatch: {count_unmatch}")

{'taxonomy_account_name': 'สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธรรมผ่านกำไรหรือขาดทุน', 'ai_account_code': '', 'prod_account_code': '100690', 'is_match': False}
{'taxonomy_account_name': 'รวมสินทรัพย์', 'ai_account_code': '', 'prod_account_code': '109999', 'is_match': False}
{'taxonomy_account_name': 'ตราสารหนี้ที่ออกและเงินกู้ยืม', 'ai_account_code': '', 'prod_account_code': '202090', 'is_match': False}
{'taxonomy_account_name': 'องค์ประกอบอื่นของส่วนของเจ้าของ', 'ai_account_code': '', 'prod_account_code': '319990', 'is_match': False}
{'taxonomy_account_name': 'รวมส่วนของเจ้าของ', 'ai_account_code': '', 'prod_account_code': '309998', 'is_match': False}
{'taxonomy_account_name': 'กำไรจากการดำเนินงานก่อนภาษีเงินได้', 'ai_account_code': '', 'prod_account_code': '510200', 'is_match': False}
{'taxonomy_account_name': 'ผลขาดทุนด้านเครดิตที่คาดว่าจะเกิดขึ้น', 'ai_account_code': '', 'prod_account_code': '510400', 'is_match': False}
{'taxonomy_account_name': '(กําไร) ขาดทุนจากการแปลงค